In [1]:
from facebook_scraper import FacebookScraper

def get_zuck_profile():
    # Initialize the scraper
    scraper = FacebookScraper()
    
    try:
        # Get profile information
        profile = scraper.get_profile("zuck")
        
        # Print key profile information
        print("Profile Information:")
        print("-" * 50)
        
        if profile.get("Name"):
            print(f"Name: {profile['Name']}")
        
        if profile.get("Friend_count"):
            print(f"Friends: {profile['Friend_count']}")
            
        if profile.get("Follower_count"):
            print(f"Followers: {profile['Follower_count']}")
            
        if profile.get("Following_count"):
            print(f"Following: {profile['Following_count']}")
            
        if profile.get("id"):
            print(f"Profile ID: {profile['id']}")
            
        if profile.get("profile_picture"):
            print(f"Profile Picture URL: {profile['profile_picture']}")
            
        if profile.get("cover_photo"):
            print(f"Cover Photo URL: {profile['cover_photo']}")
            
        # Print any additional information
        for key, value in profile.items():
            if key not in ["Name", "Friend_count", "Follower_count", "Following_count", 
                          "id", "profile_picture", "cover_photo"]:
                print(f"{key}: {value}")
                
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("\nNote: You may need to provide cookies to access this profile.")
        print("Try: scraper.get_profile('zuck', cookies='cookies.txt')")

if __name__ == "__main__":
    get_zuck_profile()

ImportError: attempted relative import with no known parent package

In [3]:
# facebook_utils.py
import itertools
import logging
import re
from urllib.parse import urljoin, parse_qs, urlparse, unquote
import warnings
import json
from datetime import datetime
import os
from bs4 import BeautifulSoup
from requests_html import HTMLSession

# Constants
FB_BASE_URL = "https://facebook.com"
FB_MOBILE_BASE_URL = "https://m.facebook.com"
FB_W3_BASE_URL = "https://www.facebook.com"
FB_MBASIC_BASE_URL = "https://mbasic.facebook.com"
DEFAULT_PAGE_LIMIT = 10

class FacebookScraper:
    """Class for creating FacebookScraper Iterators"""

    base_url = FB_MOBILE_BASE_URL
    default_headers = {
        "Accept": "*/*",
        "Connection": "keep-alive",
        "Accept-Encoding": "gzip,deflate",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15",
    }
    have_checked_locale = False

    def __init__(self, session=None, requests_kwargs=None):
        if session is None:
            session = HTMLSession()
            session.headers.update(self.default_headers)

        if requests_kwargs is None:
            requests_kwargs = {}

        self.session = session
        self.requests_kwargs = requests_kwargs
        self.request_count = 0
        self.mbasic_headers = None

    def get_profile(self, account, **kwargs):
        """Get profile information for a given Facebook account"""
        account = account.replace("profile.php?id=", "")
        result = {}

        try:
            response = self.get(f"/{account}")
            
            # Basic profile information
            try:
                result["Name"] = response.html.find("title", first=True).text.split(" | ")[0]
            except:
                result["Name"] = None

            try:
                # Friend count
                friend_elem = response.html.find("a[data-store*='friends']>div>div", first=True)
                if friend_elem:
                    result["Friend_count"] = self._parse_int(friend_elem.text.split()[0])
            except:
                result["Friend_count"] = None

            try:
                # Follower count
                follower_elem = response.html.find("div[data-sigil*='profile-intro-card-log']", 
                                                 containing="Followed by", 
                                                 first=True)
                if follower_elem:
                    result["Follower_count"] = self._parse_int(follower_elem.text)
            except:
                result["Follower_count"] = None

            # Profile and cover photos
            try:
                photo_links = response.html.find("a[href^='/photo.php']")
                if len(photo_links) >= 1:
                    result["profile_picture"] = photo_links[0].attrs.get("href")
                if len(photo_links) >= 2:
                    result["cover_photo"] = photo_links[1].attrs.get("href")
            except:
                pass

        except Exception as e:
            print(f"Error fetching profile: {str(e)}")
            return None

        return result

    def get(self, url, **kwargs):
        """Make a GET request"""
        try:
            if not url.startswith("http"):
                url = urljoin(self.base_url, url)
            
            response = self.session.get(url, **self.requests_kwargs, **kwargs)
            response.raise_for_status()
            return response
        except Exception as e:
            print(f"Error making request: {str(e)}")
            raise

    def _parse_int(self, string):
        """Helper to parse integer from string"""
        return int(''.join(filter(str.isdigit, string)))

def main():
    scraper = FacebookScraper()
    profile = scraper.get_profile("Elon R. Musk")
    print("Profile information:")
    print(json.dumps(profile, indent=2))

if __name__ == "__main__":
    main()

Profile information:
{
  "Name": "Facebook"
}


In [3]:
import time
import json
import logging
from typing import Dict, Any
from urllib.parse import urljoin
import re
from requests_html import HTMLSession
from bs4 import BeautifulSoup

class ModernFacebookScraper:
    """Modern Facebook scraper focused on detailed profile information"""
    
    BASE_URL = "https://www.facebook.com"
    MOBILE_URL = "https://m.facebook.com"
    
    def __init__(self, cookies_file: str = None):
        self.session = HTMLSession()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.9',
            'Sec-Fetch-Site': 'none',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-User': '?1',
            'Sec-Fetch-Dest': 'document',
            'Upgrade-Insecure-Requests': '1'
        })
        
        if cookies_file:
            self.load_cookies(cookies_file)

    def load_cookies(self, cookies_file: str) -> None:
        """Load cookies from file"""
        try:
            with open(cookies_file, 'r') as f:
                cookies = json.load(f)
                for cookie in cookies:
                    self.session.cookies.set(cookie['name'], cookie['value'])
        except Exception as e:
            logging.error(f"Error loading cookies: {e}")
            raise

    def get_profile_details(self, username: str) -> Dict[Any, Any]:
        profile_data = {
            'basic_info': {},
            'about': {},
            'work_education': {},
            'places': {},
            'contact': {},
            'relationships': {},
            'bio': {}
        }

        try:
            profile_url = f"{self.MOBILE_URL}/{username}/about"  # Use m.facebook.com for better parsing
            response = self.session.get(profile_url)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract Name (Improved)
            title_tag = soup.find('title')
            if title_tag:
                name_match = re.search(r"(.*?) \| Facebook", title_tag.text)  # More robust regex
                if name_match:
                  profile_data['basic_info']['name'] = name_match.group(1).strip()

            # Extract Profile Picture (Improved)
            image_tag = soup.find('img', alt=re.compile(r"^Profile picture of")) #More robust way to find profile picture
            if image_tag and 'src' in image_tag.attrs:
                profile_data['basic_info']['profile_picture'] = image_tag['src']

            # Extract About Sections (More Robust)
            about_sections = soup.find_all('div', {'class': re.compile(r'^[a-zA-Z0-9_-]+_(\d+)')}) #More robust way to find about sections

            for section in about_sections:
                header = section.find('h3') #look for header tag
                if not header:
                    header = section.find('strong') #if h3 not found look for strong tag
                if header:
                    title = header.text.strip()
                    if "Work" in title or "Education" in title:
                        profile_data['work_education'].update(self._extract_items(section, ['experience']))
                    elif "Places" in title:
                        profile_data['places'].update(self._extract_items(section, ['itemContent']))
                    elif "Contact" in title:
                        profile_data['contact'].update(self._extract_contact_info(section))
                    elif "Basic Info" in title:
                        profile_data['about'].update(self._extract_items(section, ['itemContent']))
                    elif "Relationship" in title:
                        profile_data['relationships'].update(self._extract_items(section, ['itemContent']))
                    elif "Bio" in title or "Intro" in title:  # Handle Bio or Intro
                        profile_data['bio']['text'] = section.get_text(strip=True) #Get bio text

            return profile_data

        except Exception as e:
            logging.error(f"Error fetching profile details: {e}")
            return profile_data

    def _extract_items(self, section, classes):
        data = {}
        try:
            items = section.find_all('div', {'class': re.compile(r'|'.join(classes))})
            for item in items:
                text = item.get_text(strip=True)
                if ":" in text:
                    key, value = text.split(":", 1)
                    data[key.strip()] = value.strip()
                else:
                    data.setdefault('items', []).append(text)
        except Exception as e:
            logging.error(f"Error extracting items: {e}")
        return data

    def _extract_contact_info(self, section):
        data = {}
        try:
            items = section.find_all('div', {'class': re.compile(r'itemContent')})
            for item in items:
                text = item.get_text(strip=True)
                parts = text.split(":", 1)
                if len(parts) == 2:
                    key, value = parts
                    data[key.strip()] = value.strip()
                elif len(parts) == 1:
                    data.setdefault('items', []).append(parts[0])
        except Exception as e:
            logging.error(f"Error extracting contact info: {e}")
        return data

    def _extract_basic_info(self, section) -> dict:
        """Extract basic information"""
        data = {}
        try:
            items = section.find_all('div', {'class': 'itemContent'})
            for item in items:
                text = item.get_text().strip()
                if ':' in text:
                    key, value = text.split(':', 1)
                    data[key.strip()] = value.strip()
        except Exception as e:
            logging.error(f"Error extracting basic info: {e}")
        return data

    def _extract_relationships(self, section) -> dict:
        """Extract relationship information"""
        data = {}
        try:
            items = section.find_all('div', {'class': 'itemContent'})
            for item in items:
                text = item.get_text().strip()
                if ':' in text:
                    key, value = text.split(':', 1)
                    data[key.strip()] = value.strip()
        except Exception as e:
            logging.error(f"Error extracting relationships: {e}")
        return data

def main():
    # Example usage
    scraper = ModernFacebookScraper(cookies_file='cookies.json')
    profile_info = scraper.get_profile_details('Elon R. Musk ')
    print(json.dumps(profile_info, indent=2))

if __name__ == "__main__":
    main()

ERROR:root:Error fetching profile details: 502 Server Error: Bad Gateway for url: https://m.facebook.com/Elon%20R.%20Musk%20/about


{
  "basic_info": {},
  "about": {},
  "work_education": {},
  "places": {},
  "contact": {},
  "relationships": {},
  "bio": {}
}


In [ ]:
EAAIG4eOZAdEUBO8EGQZCb8WZAr6Rbi5YZBgAoWEY9yCdhueNbaCDnheEKPKBCzGlTiGdpsilA7D5XnTMozuFt8aEIBRize8ZBRZBZAPaNkEhMSIfsl7KqNPP3vhJNIZAc8U84Wa1lO4t2pD9YLw37nRcLTQeNQ8sCILKfHu1Y4TgyMHIQlxg0zXK3vK9TZB1C4Be6ZCUkBGZBbz35ZCVu2a3irwbQTJVnK7cDUXHvsKqUv0wEZBb5lfzczLTEPagKVCnXzC8ANey06gZDZD

In [7]:
import requests
import json
import ollama

# Function to get Facebook profile content using the Graph API
def get_facebook_profile_data(access_token, user_id="Mark Zuckerberg"):
    url = f"https://graph.facebook.com/v12.0/{user_id}?fields=name,bio,interests&access_token={access_token}"
    response = requests.get(url)
    data = response.json()
    
    if "error" in data:
        print(f"Error retrieving data: {data['error']['message']}")
        return None
    
    # Return relevant content for pickup line generation
    profile_content = {
        "name": data.get("name", "Unknown"),
        "bio": data.get("bio", "No bio available."),
        "interests": data.get("interests", "No interests listed.")
    }
    
    return profile_content

# Function to generate pickup line using the ollama LLM
def generate_pickup_line(profile_content):
    # Combine profile details into a prompt for the LLM
    prompt = f"Generate a fun and engaging pickup line based on the following Facebook profile:\nName: {profile_content['name']}\nBio: {profile_content['bio']}\nInterests: {profile_content['interests']}\nPickup Line:"
    
    # Initialize ollama client and generate response
    client = ollama.Ollama()
    response = client.chat(model="llama2", messages=[{"role": "user", "content": prompt}])
    
    # Extract the generated pickup line from the response
    pickup_line = response['message']['content']
    return pickup_line

# Main function to fetch profile and generate a pickup line
def main():
    # access_token = input("Enter your Facebook access token: ")
    access_token = "EAAIG4eOZAdEUBO8EGQZCb8WZAr6Rbi5YZBgAoWEY9yCdhueNbaCDnheEKPKBCzGlTiGdpsilA7D5XnTMozuFt8aEIBRize8ZBRZBZAPaNkEhMSIfsl7KqNPP3vhJNIZAc8U84Wa1lO4t2pD9YLw37nRcLTQeNQ8sCILKfHu1Y4TgyMHIQlxg0zXK3vK9TZB1C4Be6ZCUkBGZBbz35ZCVu2a3irwbQTJVnK7cDUXHvsKqUv0wEZBb5lfzczLTEPagKVCnXzC8ANey06gZDZD"
    
    # Get the user's Facebook profile content
    profile_content = get_facebook_profile_data(access_token)
    
    if profile_content:
        # Generate a pickup line based on the profile content
        pickup_line = generate_pickup_line(profile_content)
        print(f"Generated Pickup Line: {pickup_line}")
    else:
        print("Could not retrieve profile data.")

if __name__ == "__main__":
    main()

Error retrieving data: Unsupported get request. Object with ID 'Mark Zuckerberg' does not exist, cannot be loaded due to missing permissions, or does not support this operation. Please read the Graph API documentation at https://developers.facebook.com/docs/graph-api
Could not retrieve profile data.


In [11]:
access_token = "EAAIG4eOZAdEUBO8EGQZCb8WZAr6Rbi5YZBgAoWEY9yCdhueNbaCDnheEKPKBCzGlTiGdpsilA7D5XnTMozuFt8aEIBRize8ZBRZBZAPaNkEhMSIfsl7KqNPP3vhJNIZAc8U84Wa1lO4t2pD9YLw37nRcLTQeNQ8sCILKfHu1Y4TgyMHIQlxg0zXK3vK9TZB1C4Be6ZCUkBGZBbz35ZCVu2a3irwbQTJVnK7cDUXHvsKqUv0wEZBb5lfzczLTEPagKVCnXzC8ANey06gZDZD"

In [14]:
# Adjusting the function for a public Facebook page
def get_facebook_page_data(access_token, page_id="markzuckerberg"):
    url = f"https://graph.facebook.com/v12.0/{page_id}?fields=name,about,fan_count&access_token={access_token}"
    response = requests.get(url)
    data = response.json()
    
    if "error" in data:
        print(f"Error retrieving data: {data['error']['message']}")
        return None
    
    # Return relevant content for pickup line generation
    page_content = {
        "name": data.get("name", "Unknown"),
        "about": data.get("about", "No information available."),
        "fan_count": data.get("fan_count", "No fan count available.")
    }
    
    return page_content
page_data = get_facebook_page_data(access_token=access_token, page_id="zuck")


Error retrieving data: Unsupported get request. Object with ID 'zuck' does not exist, cannot be loaded due to missing permissions, or does not support this operation. Please read the Graph API documentation at https://developers.facebook.com/docs/graph-api
